In [ ]:
# github url: https://www.github.com/attentionmech/mav
# report any issue to: https://x.com/@attentionmech

!pip install -q git+https://www.github.com/attentionmech/mav
!mav --help

  Preparing metadata (setup.py) ... done
usage: mav [-h] [--model MODEL] [--prompt PROMPT] [--max-new-tokens MAX_NEW_TOKENS]
           [--aggregation {l2,max_abs}] [--refresh-rate REFRESH_RATE] [--interactive]
           [--device {cpu,cuda,mps}] [--scale {linear,log,minmax}] [--limit-chars LIMIT_CHARS]
           [--temp TEMP] [--top-k TOP_K] [--top-p TOP_P] [--min-p MIN_P]
           [--repetition-penalty REPETITION_PENALTY] [--backend {transformers}] [--seed SEED]
           [--max-bar-length MAX_BAR_LENGTH]
           [--selected-panels SELECTED_PANELS [SELECTED_PANELS ...]]
           [--num-grid-rows NUM_GRID_ROWS] [--version]

Model Activation Visualizer

options:
  -h, --help            show this help message and exit
  --model MODEL         Hugging Face model name (default: gpt2)
  --prompt PROMPT       Initial prompt for text generation
  --max-new-tokens MAX_NEW_TOKENS
                        Number of tokens to generate
  --aggregation {l2,max_abs}
                        

In [ ]:
from openmav.mav import MAV
from openmav.view.panels.panel_base import PanelBase
import random



class RandomBars(PanelBase):
    def __init__(self, measurements ,max_bar_length=20, limit_chars=50):
        super().__init__("Random Bars", "green", max_bar_length, limit_chars)
        self.measurements=measurements

    def get_panel_content(self):
        bars_str = ""
        for i in range(5):
            value = random.uniform(-1, 1)
            bar_length = int(abs(value) * self.max_bar_length)
            bar = "█" * bar_length
            color = "cyan" if value >= 0 else "red"
            bars_str += f"[bold white]Bar {i:2d}[/] | [{color}]{bar.ljust(self.max_bar_length)}[/] [bold yellow]{value:+.2f}[/]\n"
        return bars_str

MAV("gpt2", "hello world ", num_grid_rows=1, selected_panels=["generated_text", "RandomBars"], external_panels=[RandomBars], max_new_tokens=5)

Output()

In [ ]:
# this class is from openmav.api.model.measurements.ModelMeasurements
# this is what you receive from backend for your panel
#
# class ModelMeasurements:
#     mlp_activations: torch.Tensor
#     mlp_normalized: torch.Tensor
#     entropy_values: torch.Tensor //attention
#     entropy_normalized: torch.Tensor //attention
#     generated_text: str
#     predicted_char: str
#     next_token_probs: torch.Tensor
#     top_ids: torch.Tensor
#     top_probs: torch.Tensor
#     logits: torch.Tensor
#     decoded_tokens: List[str]

class EntropyFire(PanelBase):
    def __init__(self, measurements,max_bar_length=20, limit_chars=50):
        super().__init__("Entropy Fire", "red",max_bar_length=max_bar_length, limit_chars=limit_chars)
        self.measurements = measurements

    def get_panel_content(self):
        fire_str = ""
        entropy_vals = self.measurements.entropy_values[:10]  # Take first 10 entropy values

        for i, entropy in enumerate(entropy_vals):
            intensity = min(int(entropy ), 9)  # Normalize entropy to a scale of 0-9
            fire_colors = ["blue", "cyan", "green", "yellow", "orange", "red", "bright_red"]
            color = fire_colors[intensity%len(fire_colors)]
            fire_str += f"[{color}]🔥[/]" * (intensity + 1) + "\n"

        return fire_str

MAV("gpt2", "hello world", num_grid_rows=1, selected_panels=["generated_text", "EntropyFire"], external_panels=[EntropyFire], max_new_tokens=10)


Output()